## Outliers (By State)

wip

In [1]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import chi2, zscore
from sklearn.decomposition import PCA
from pandas.plotting import scatter_matrix

os.getcwd()

'/home/rc/capstone/notebooks/data-cleaning'

In [8]:
# Checking env/venv dependencies. Install missing dependencies here.
%pip install scipy
import scipy
print("scipy", scipy.__version__)

%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
scipy 1.16.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [2]:
os.chdir('../../data')
!ls

Key_Indicator_State_and_District_wise_data  Key_indicator_statewise.csv
Key_Indicators_Statewise.sqlite		    healthanalytics.zip
Key_indicator_districtwise.csv		    statewise.sqlite


In [3]:
# Load statewise dataset
df = pd.read_csv('Key_indicator_statewise.csv')

# Remove all letter prefixes eg AA_ - ZZ_ from feature columns
df.columns = [re.sub(r'^[A-Z]{2}_', '', col) for col in df.columns]

# Check all features in resulting dataframe
df.head(10) # 9 states surveyed

,State_Name,Sample_Units_Total,Sample_Units_Rural,Sample_Units_Urban,Households_Total,Households_Rural,Households_Urban,Population_Total,Population_Rural,Population_Urban,...,Under_Five_Mortality_Rate_U5MR_Rural_Lower_Limit,Under_Five_Mortality_Rate_U5MR_Rural_Upper_Limit,Under_Five_Mortality_Rate_U5MR_Urban_Lower_Limit,Under_Five_Mortality_Rate_U5MR_Urban_Upper_Limit,Sex_Ratio_At_Birth_Total_Lower_Limit,Sex_Ratio_At_Birth_Total_Upper_Limit,Sex_Ratio_At_Birth_Rural_Lower_Limit,Sex_Ratio_At_Birth_Rural_Upper_Limit,Sex_Ratio_At_Birth_Urban_Lower_Limit,Sex_Ratio_At_Birth_Urban_Upper_Limit
0,Assam,1784,1412,372,388853,319766,69087,1809610,1518639,290971,...,76,79,33,40,936,958,930,954,945,1009
1,Bihar,2356,1981,375,612684,568030,44654,3227867,2993906,233961,...,71,73,48,55,918,932,918,933,893,953
2,Chhattisgarh,1255,926,329,287085,226554,60531,1264309,994416,269893,...,63,67,37,43,943,969,944,973,916,975
3,Jharkhand,2108,1513,595,392734,318142,74592,2019298,1644036,375262,...,56,58,27,31,920,940,932,954,860,909
4,Madhya Pradesh,2557,1660,897,519811,357179,162632,2389787,1629355,760432,...,91,94,55,59,897,914,906,926,860,893
5,Odisha,2364,1798,566,477065,392913,84152,1992799,1637152,355647,...,78,81,45,51,897,919,895,918,889,943
6,Rajasthan,1841,1294,547,362671,300306,62365,1828116,1521017,307099,...,79,82,48,55,877,897,875,896,869,920
7,Uttar Pradesh,3927,2782,1145,883613,710313,173300,4808503,3883293,925210,...,96,98,61,65,915,928,922,935,874,904
8,Uttarakhand,2501,1962,539,392643,308474,84169,1726477,1365247,361230,...,50,53,34,40,861,884,868,894,823,877


In [4]:
# Check statwise csv data shape
df.shape

(9, 643)

In [5]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
State_Name,9,9,Assam,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sample_Units_Total,9.0,NaN,NaN,NaN,2299.222222,738.802372,1255.0,1841.0,2356.0,2501.0,3927.0
Sample_Units_Rural,9.0,NaN,NaN,NaN,1703.111111,525.746004,926.0,1412.0,1660.0,1962.0,2782.0
Sample_Units_Urban,9.0,NaN,NaN,NaN,596.111111,266.659635,329.0,375.0,547.0,595.0,1145.0
Households_Total,9.0,NaN,NaN,NaN,479684.333333,179108.137044,287085.0,388853.0,392734.0,519811.0,883613.0
...,...,...,...,...,...,...,...,...,...,...,...
Sex_Ratio_At_Birth_Total_Upper_Limit,9.0,NaN,NaN,NaN,926.777778,27.160531,884.0,914.0,928.0,940.0,969.0
Sex_Ratio_At_Birth_Rural_Lower_Limit,9.0,NaN,NaN,NaN,910.0,26.167728,868.0,895.0,918.0,930.0,944.0
Sex_Ratio_At_Birth_Rural_Upper_Limit,9.0,NaN,NaN,NaN,931.444444,26.524098,894.0,918.0,933.0,954.0,973.0
Sex_Ratio_At_Birth_Urban_Lower_Limit,9.0,NaN,NaN,NaN,881.0,35.298725,823.0,860.0,874.0,893.0,945.0


In [6]:
# Check for null counts per feature overall
null_counts = df.isnull().sum()

print("Null counts per feature:")
print(null_counts[null_counts > 0].sum()) 

Null counts per feature:
0


In [ ]:
# Univariate outlier tests/checks
# Specify a feature column here to test/check for outliers
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)]

In [8]:
# Iterate through numeric columns and compute IQR-based outliers
numeric_cols = df.select_dtypes(include='number').columns

outlier_summary = {}

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]

    outlier_summary[col] = {
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "num_outliers": outliers.shape[0],
        "percent_outliers": 100 * outliers.shape[0] / df.shape[0]
    }

outlier_df = pd.DataFrame(outlier_summary).T
outlier_df.sort_values("percent_outliers", ascending=False)

,lower_bound,upper_bound,num_outliers,percent_outliers
Average_Month_By_Which_Children_Received_Foods_Other_Than_Breast_Milk_Semi_Solid_Mashed_Food_Urban,7.10,7.58,4.0,44.444444
Children_Who_Have_Received_Polio_Dose_At_Birth_Urban,83.30,91.62,4.0,44.444444
Work_Participation_Rate_15_Years_And_Above_Person_Urban,41.35,46.87,4.0,44.444444
Children_With_Birth_Weight_Less_Than_2_5_Kg_Urban,19.90,28.70,3.0,33.333333
Children_Aged_5_14_Years_Engaged_In_Work_Person_Rural,2.57,3.45,3.0,33.333333
...,...,...,...,...
Persons_Suffering_From_Acute_Illness_And_Taking_Treatment_From_Any_Source_Person_Total,84.50,106.10,0.0,0.000000
Persons_Suffering_From_Acute_Illness_Per_100000_Population_Any_Type_Of_Acute_Illness_Female_Urban,2274.50,16318.50,0.0,0.000000
Sex_Ratio_At_Birth_Rural_Upper_Limit,864.00,1008.00,0.0,0.000000
Sex_Ratio_At_Birth_Rural_Lower_Limit,842.50,982.50,0.0,0.000000


In [9]:
# Z-score/Standard deviation rule (range -3 to 3 std from the mean)
df['z'] = zscore(df[col].dropna())
outliers = df[df['z'].abs() > 3]
outliers

,State_Name,Sample_Units_Total,Sample_Units_Rural,Sample_Units_Urban,Households_Total,Households_Rural,Households_Urban,Population_Total,Population_Rural,Population_Urban,...,Under_Five_Mortality_Rate_U5MR_Rural_Upper_Limit,Under_Five_Mortality_Rate_U5MR_Urban_Lower_Limit,Under_Five_Mortality_Rate_U5MR_Urban_Upper_Limit,Sex_Ratio_At_Birth_Total_Lower_Limit,Sex_Ratio_At_Birth_Total_Upper_Limit,Sex_Ratio_At_Birth_Rural_Lower_Limit,Sex_Ratio_At_Birth_Rural_Upper_Limit,Sex_Ratio_At_Birth_Urban_Lower_Limit,Sex_Ratio_At_Birth_Urban_Upper_Limit,z


In [ ]:
# Multivariate Outlier Detection
#iso = IsolationForest(contamination=0.05)
#print(df['outlier_flag'] == iso.fit_predict(df.select_dtypes('number')))

In [11]:
num = df.select_dtypes(include=np.number).copy()
Z = pd.DataFrame(
    zscore(num.values, nan_policy="omit", axis=0),  # ndarray same shape as 'num'
    index=num.index,
    columns=num.columns
)

In [12]:
mask = Z.abs() > 2.5 (std range -3 to 3)
outlier_counts = mask.sum().sort_values(ascending=False)
rows_with_any_outlier = df.loc[mask.any(axis=1)]
rows_with_any_outlier

,State_Name,Sample_Units_Total,Sample_Units_Rural,Sample_Units_Urban,Households_Total,Households_Rural,Households_Urban,Population_Total,Population_Rural,Population_Urban,...,Under_Five_Mortality_Rate_U5MR_Rural_Upper_Limit,Under_Five_Mortality_Rate_U5MR_Urban_Lower_Limit,Under_Five_Mortality_Rate_U5MR_Urban_Upper_Limit,Sex_Ratio_At_Birth_Total_Lower_Limit,Sex_Ratio_At_Birth_Total_Upper_Limit,Sex_Ratio_At_Birth_Rural_Lower_Limit,Sex_Ratio_At_Birth_Rural_Upper_Limit,Sex_Ratio_At_Birth_Urban_Lower_Limit,Sex_Ratio_At_Birth_Urban_Upper_Limit,z
0,Assam,1784,1412,372,388853,319766,69087,1809610,1518639,290971,...,79,33,40,936,958,930,954,945,1009,1.942926
5,Odisha,2364,1798,566,477065,392913,84152,1992799,1637152,355647,...,81,45,51,897,919,895,918,889,943,0.289490
7,Uttar Pradesh,3927,2782,1145,883613,710313,173300,4808503,3883293,925210,...,98,61,65,915,928,922,935,874,904,-0.687540


In [13]:
dups = num.columns[num.columns.duplicated()]
print("Duplicate columns count:", len(dups))

Duplicate columns count: 0
